In [1]:
class TooManyMatchesError(Exception):
    pass

class SymbolNotFoundError(Exception):
    pass

class SymbolStore:
    def __init__(self):
        self.symbols = list()
    
    def _get_by(self, **kwargs):
        for s in self.symbols:
            for k, v in kwargs.items():
                if s.get(k, None) != v:
                    break
            else:
                yield s


    def get_first(self, **kwargs):
        try:
            return next(self._get_by(**kwargs))
        except StopIteration as exc:
            raise SymbolNotFoundError("Symbol not found") from exc
    
    def get_one(self, **kwargs):
        i = self._get_by(**kwargs)
        
        try:
            s = next(i)
        except StopIteration as exc:
            raise SymbolNotFoundError("Symbol not found") from exc
        
        try:
            next(i)
        except StopIteration:
            return s
        else:
            raise TooManyMatchesError("More than one match while calling `get_one`")
    
    def get_all(self, **kwargs):
        return list(self._get_by(**kwargs))
    
    def get_or_create(self, **kwargs):
        try:
            s = self.get_one(**kwargs)
        except SymbolNotFoundError:
            s = kwargs.copy()
            self.symbols.append(s)
        return s

In [2]:
import regex
from itertools import groupby
from uuid import uuid4

class Token(str):    
    def __repr__(self):
        return f"{self.__class__.__name__}({super().__repr__()})"
        
class Word(Token):
    pass

class Acronym(Word):
    pass

class Separator(Token):
    pass

class Name:
    _camel_case_regex = regex.compile(r"^([a-z]+)((:?[A-Z0-9][A-Za-z0-9]*?)*)$")
    _full_camel_case_regex = regex.compile(r"^([A-Z][a-z0-9A-Z]*?)((:?[A-Z0-9][A-Za-z0-9]*?)*)$")
    
    def __init__(self, *tokens, **meta):
        self._tokens = list(tokens)
        self._meta = meta

    @classmethod
    def from_snake_case_all_caps(cls, text):
        tokens = []
        for t in text.split('_'):
            tokens.append(Word(t))
            tokens.append(Separator('_'))
            
        return cls(*tokens[:-1], original=text)

    @staticmethod
    def _scan_camel_case_regex(regex, text):
        tokens = []
        if (match := regex.match(text)) is not None:
            captures = [match.group(1)] + list(match.captures(3))
            for _, ws in groupby(captures, key=lambda k: str(k.isalpha()) if len(k) == 1 else repr(uuid4())):
                group = list(ws)
                if len(group) > 1:
                    tokens.append(Acronym(''.join(group)))
                else:
                    tokens.append(Word(group[0]))
            return tokens
        else:
            raise ValueError("Not in camel case format")

    @classmethod
    def from_camel_case(cls, text):
        return cls(*cls._scan_camel_case_regex(cls._camel_case_regex, text), original=text)
    
    @classmethod
    def from_full_camel_case(cls, text):
        return cls(*cls._scan_camel_case_regex(cls._full_camel_case_regex, text), original=text)

    def to_snake_case(self):
        words = []
        for t in self._tokens:
            if isinstance(t, Word):
                words.append(t.lower())
        return '_'.join(words)
    
    def to_snake_case_all_caps(self):
        return self.to_snake_case().upper()
    
    def to_camel_case(self):
        words = []
        is_first_word = True
        for t in self._tokens:
            if isinstance(t, Word):
                if is_first_word:
                    words.append(t.lower())
                    is_first_word = False
                else:
                    words.append(t.capitalize())
        return ''.join(words)        
    
    def to_full_camel_case(self):
        return ''.join(t.capitalize() for t in self._tokens if isinstance(t, Word))

    def __repr__(self):
        return f"Name(*{repr(self._tokens)}, **{repr(self._meta)})"

In [3]:
class Module:
    """A PS module"""
    def __init__(self, declarations=None):
        if declarations is None:
            declarations = list()
        self.declarations = declarations  # [Definition]

class Definition:
    def render_with_name(self, name):
        raise NotImplementedError("To be implemented in subclass")
    def __repr__(self):
        return f"{self.__class__.__name__}({self.name})"

class UnionType(Definition):
    def __init__(self, name, constructors=None):
        self.name = name
        if constructors is None:
            constructors = list()
        self.constructors = constructors  # [name]

class Function(Definition):
    """A PS function"""
    def __init__(self, name, body_template):
        self.name = name
        self.parameters = dict()  # name: type_resolver
        self.result_type = None  # type_resolver
        self.pointfree_parameter_types = list()  # [type_resolver]
        self.body_template = body_template  # jinja2 template to implement this function

class ForeignFunctionImport(Definition):
    def __init__(self, name, js_template):
        self.name = name
        self.parameter_types = list()
        self.result_type = None
        self.js_template = js_template

class ForeignTypeImport(Definition):
    def __init__(self, name, js_template):
        self.name = name
        self.js_template = js_template

In [4]:
import json

official_api = SymbolStore()

data = json.load(open('gasapi.json'))

for d in data:
    s = official_api.get_or_create(type=d['type'], url=d['url'], name=d['name'])
    s['methods'] = d['methods']
    s['properties'] = d['properties']

In [5]:
purescript_api = SymbolStore()

for enum in official_api.get_all(type='enum'):
    module = Module()
    
    # The union declaration
    union_name = enum['name']  # Already in full camel case
    constructors = [Name.from_snake_case_all_caps(p['name']).to_full_camel_case()
                    for p in enum['properties']]
    union = UnionType(union_name, constructors)
    module.declarations.append(union)
    
    # The foreign type import
    foreign_type_name = f"{union_name}Impl"
    foreign_type = ForeignTypeImport(foreign_type_name, None)
    module.declarations.append(foreign_type)
    
    for p in enum['properties']:
        foreign_fn_name = Name.from_snake_case_all_caps(p['name']).to_camel_case()
        foreign_fn = ForeignFunctionImport(foreign_fn_name, "")  # TODO: add template
        foreign_fn.result_type = lambda: foreign_type
        module.declarations.append(foreign_fn)
    
    #
    # JS2PS & PS2JS
    #
    transform_fn_prefix_name = Name.from_snake_case_all_caps(enum['name']).to_camel_case()
    
    # A foreign function to transform from a JS value to PS
    foreign_fn_js2ps_name = f"{transform_fn_prefix_name}JS2PSImpl"
    foreign_fn_js2ps = ForeignFunctionImport(foreign_fn_js2ps_name, "")  # TODO: add template
    foreign_fn_js2ps.result_type = lambda: ""  # TODO: add type
    module.declarations.append(foreign_fn_js2ps)
    
    # A wrapper for the last function
    fn_js2ps_name = f"{transform_fn_prefix_name}JS2PS"
    fn_js2ps = Function(fn_js2ps_name, "")  # TODO: add template
    fn_js2ps.pointfree_parameter_types.append(lambda: foreign_type)
    fn_js2ps.result_type = lambda: union
    module.declarations.append(fn_js2ps)

    # A function to transform from a PS value to JS
    fn_ps2js_name = f"{transform_fn_prefix_name}PS2JS"
    fn_ps2js = Function(fn_ps2js_name, "")  # TODO: add template
    fn_ps2js.pointfree_parameter_types.append(lambda: union)
    fn_ps2js.result_type = lambda: foreign_type
    module.declarations.append(fn_ps2js)
    
    # Save module
    purescript_api.get_or_create(source=enum['url'], type='module')['definition'] = module

In [6]:
purescript_api.get_all()[1]['definition'].declarations

[UnionType(BigQueryParameterType),
 ForeignTypeImport(BigQueryParameterTypeImpl),
 ForeignFunctionImport(string),
 ForeignFunctionImport(int64),
 ForeignFunctionImport(bool),
 ForeignFunctionImport(float64),
 ForeignFunctionImport(bigqueryparametertypeJS2PSImpl),
 Function(bigqueryparametertypeJS2PS),
 Function(bigqueryparametertypePS2JS)]